## Thank you very much for raising the rating of this notebook. I hope it helps beginners. If it is useful to you, please up this)

In [ ]:

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Load the libraries


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.pipeline import Pipeline
import re, spacy, string
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

Load data

In [ ]:
train_df = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')
test_df = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/test.csv.zip')
train_df.head()

Check miss value

In [ ]:
train_df.isnull().sum()

Check values != 0 or 1

In [ ]:
print(train_df.loc[(train_df.toxic!=0)&(train_df.toxic!=1)].shape[0])
print(train_df.loc[(train_df.severe_toxic!=0)&(train_df.severe_toxic!=1)].shape[0])
print(train_df.loc[(train_df.obscene!=0)&(train_df.obscene!=1)].shape[0])
print(train_df.loc[(train_df.threat!=0)&(train_df.threat!=1)].shape[0])
print(train_df.loc[(train_df.insult!=0)&(train_df.insult!=1)].shape[0])
print(train_df.loc[(train_df.identity_hate!=0)&(train_df.identity_hate!=1)].shape[0])

Look at amount positive points in class

In [ ]:
print(train_df.loc[train_df.toxic==1].shape[0])
print(train_df.loc[train_df.severe_toxic==1].shape[0])
print(train_df.loc[train_df.obscene==1].shape[0])
print(train_df.loc[train_df.threat==1].shape[0])
print(train_df.loc[train_df.insult==1].shape[0])
print(train_df.loc[train_df.identity_hate==1].shape[0])

Create y

In [ ]:
y = train_df.drop(['id','comment_text'],axis = 1)
y

Look within Seaborn our labels

In [ ]:
column_dict = {}
for i in list(y.columns):
    column_dict[i]=y[i].sum()

column_list = sorted(column_dict,key=column_dict.get,reverse=True)

In [ ]:
plt.figure(figsize=(12,8))
sns.barplot(column_list,y.sum().sort_values(ascending=False))
plt.xticks(rotation=45)
plt.show()

Our text

In [ ]:
train_df['comment_text'].head()

Load standart stopwords from nltk
### Note: for better result, think about this words, look their and maybe we have to remove some of them 

In [ ]:
stopwords_ = stopwords.words('english')
stopwords_[:10]

Let's clear our text from wrong words, links, etc.

Also add stemming


In [ ]:
stemmer = SnowballStemmer(language='english')
def simple_clear(text):
    text = text.lower()
    text = re.sub("[^a-zA-Z]", ' ', text)
    text = re.sub('<.*?>','',text)
    text = re.sub(r"https?:\/\/t.co\/[A-Za-z0-9]+", "", text)
    text = re.sub('\[|\(.*\]|\)','', text) 
    text = re.sub("(\\W)"," ",text) 
    text = re.sub('\S*\d\S*\s*','', text)  
    text = text.split()
    text = [stemmer.stem(word) for word in text if not word in set(stopwords_)]
    text = ' '.join(text)
    return text

For good speed perform cleaning text at df.apply()

In [ ]:
train_df['clean'] = train_df['comment_text'].apply(simple_clear)
train_df['clean'][:5]

Create X raw data

In [ ]:
X_raw = train_df['clean']
X_train, X_test, y_train, y_test = train_test_split(X_raw, y, test_size=0.2, random_state=42,shuffle = True)

Look amount labels in our train

In [ ]:
look_train = {}
for i in list(y_train.columns):
    look_train[i]=y_train[i].sum()

column_list_train = sorted(look_train,key=look_train.get,reverse=True)

In [ ]:
plt.figure(figsize=(12,8))
sns.barplot(column_list_train,y_train.sum().sort_values(ascending=False))
plt.xticks(rotation=45)
plt.show()

Let's transform 'clear' text after remove and stemming in TF IDF

In [ ]:
word_vectorizer = TfidfVectorizer(
    strip_accents='unicode',                 
    token_pattern=r'\w{1,}',    
    ngram_range=(1, 3),         
    stop_words = stopwords_,
    sublinear_tf=True)

word_vectorizer.fit(X_train)

In [ ]:
X_train_transformed = word_vectorizer.transform(X_train)
X_test_transformed = word_vectorizer.transform(X_test)

Look at shape data

In [ ]:

print('X_train_transformed', X_train_transformed.shape)
print('y_train', y_train.shape)
print('X_test_transformed', X_test_transformed.shape)
print('y_test', y_test.shape)

In [ ]:
X_train_transformed

In [ ]:
y_train

## At this place, i think: for some reason everyone writes and does according to the metric ROC_AUC and also such a metric is required by the competition. We have a huge imbalance of classes, and the fact that the model will distinguish only 2 out of 5 classes, as it seemed to me, is not enough. Therefore, I will give research from two positions ROC_AUC and F1score , because all hyperparameters will be different depending on the choice of metric.

# Roc_auc

In [ ]:
%%time
pipe_Log_reg =  Pipeline([("Log_reg", LogisticRegression(random_state = 42,max_iter = 10000))])
params = {'Log_reg__penalty': ['l2'],
          'Log_reg__C': [0.5,0.75,1],
          'Log_reg__class_weight': ['balanced',None],
          'Log_reg__solver': ['liblinear']}
Log_reg = GridSearchCV(pipe_Log_reg,params,scoring='roc_auc',return_train_score=True,n_jobs=-1)
Log_reg.fit(X_train_transformed,y_train.toxic)
print(f'Best_score = {Log_reg.best_score_}')
print(f'Best_params = {Log_reg.best_params_}')
print(f'Roc_auc at test:  = {roc_auc_score(y_test.toxic,Log_reg.predict(X_test_transformed))}')

In [ ]:
log_reg = LogisticRegression(C = 1, penalty='l2', solver = 'liblinear',class_weight = 'balanced', random_state=42)

classifier_ovr_log = OneVsRestClassifier(log_reg)
classifier_ovr_log.fit(X_train_transformed, y_train)

In [ ]:
print("ROC AUC Score Test:", roc_auc_score(y_test, classifier_ovr_log.predict(X_test_transformed)))
print(classification_report(y_test, classifier_ovr_log.predict(X_test_transformed)))

## LogisticRegression make ROC_AUC = 0.90175, but f1_score(micro) there 0.67

Look at SVC

In [ ]:
%%time
pipe_SVC =  Pipeline([("SVC", LinearSVC(max_iter = 10000,random_state = 42))])
params = {'SVC__penalty': ['l2','l1'],
          'SVC__C': [0.25,0.5,0.75],
          'SVC__class_weight': ['balanced',None]}
SVC_ = GridSearchCV(pipe_SVC,params,scoring='roc_auc',return_train_score=True,n_jobs=-1)
SVC_.fit(X_train_transformed,y_train.toxic)
print(f'Best_score = {SVC_.best_score_}')
print(f'Best_params = {SVC_.best_params_}')
print(f'Roc_auc at test:  = {roc_auc_score(y_test.toxic,SVC_.predict(X_test_transformed))}')

In [ ]:
SVC_auc = LinearSVC(C = 0.5,class_weight = None,max_iter = 10000, random_state=42)

classifier_ovr_SVC_auc = OneVsRestClassifier(SVC_auc)
classifier_ovr_SVC_auc.fit(X_train_transformed, y_train)

In [ ]:
print("ROC AUC Score Test:", roc_auc_score(y_test, classifier_ovr_SVC_auc.predict(X_test_transformed)))
print(classification_report(y_test, classifier_ovr_SVC_auc.predict(X_test_transformed)))

## LinearSVC make ROC_AUC = 0.7356, but f1_score(micro) there 0.74

# F1_score

In [ ]:
%%time
pipe_Log_reg =  Pipeline([("Log_reg", LogisticRegression(random_state = 42,max_iter = 10000))])
params = {'Log_reg__penalty': ['l2'],
          'Log_reg__C': [0.1,0.2,0.3],
          'Log_reg__class_weight': ['balanced',None],
          'Log_reg__solver': ['liblinear']}
Log_reg = GridSearchCV(pipe_Log_reg,params,scoring='f1',return_train_score=True,n_jobs=-1)
Log_reg.fit(X_train_transformed,y_train.toxic)
print(f'Best_score = {Log_reg.best_score_}')
print(f'Best_params = {Log_reg.best_params_}')
print(f'Roc_auc at test:  = {roc_auc_score(y_test.toxic,Log_reg.predict(X_test_transformed))}')

In [ ]:
log_reg = LogisticRegression(C = 0.3, penalty='l2', solver = 'liblinear',class_weight = 'balanced', random_state=42)

classifier_ovr_log = OneVsRestClassifier(log_reg)
classifier_ovr_log.fit(X_train_transformed, y_train)

In [ ]:
print("ROC AUC Score Test:", roc_auc_score(y_test, classifier_ovr_log.predict(X_test_transformed)))
print(classification_report(y_test, classifier_ovr_log.predict(X_test_transformed)))

## LogisticRegression make ROC_AUC = 0.90555, but f1_score(micro) there 0.63

Look at SVC

In [ ]:
%%time
pipe_SVC =  Pipeline([("SVC", LinearSVC(max_iter = 10000,random_state = 42))])
params = {'SVC__penalty': ['l2','l1'],
          'SVC__C': [0.1,0.2,0.3],
          'SVC__class_weight': ['balanced',None]}
SVC_ = GridSearchCV(pipe_SVC,params,scoring='f1',return_train_score=True,n_jobs=-1)
SVC_.fit(X_train_transformed,y_train.toxic)
print(f'Best_score = {SVC_.best_score_}')
print(f'Best_params = {SVC_.best_params_}')
print(f'Roc_auc at test:  = {roc_auc_score(y_test.toxic,SVC_.predict(X_test_transformed))}')

In [ ]:
SVC_auc = LinearSVC(C = 0.2,class_weight = 'balanced',max_iter = 10000, random_state=42)

classifier_ovr_SVC_auc = OneVsRestClassifier(SVC_auc)
classifier_ovr_SVC_auc.fit(X_train_transformed, y_train)

In [ ]:
print("ROC AUC Score Test:", roc_auc_score(y_test, classifier_ovr_SVC_auc.predict(X_test_transformed)))
print(classification_report(y_test, classifier_ovr_SVC_auc.predict(X_test_transformed)))

## LinearSVC make ROC_AUC = 0.8753, but f1_score(micro) there 0.71

# Inference:
### if we need max ROC_AUC, then use:
log_reg = LogisticRegression(C = 1, penalty='l2', solver = 'liblinear',class_weight = 'balanced', random_state=42)

classifier_ovr_log = OneVsRestClassifier(log_reg)

### if we need max f1_score, then use:
SVC_auc = LinearSVC(C = 0.5,class_weight = None,max_iter = 10000, random_state=42)

classifier_ovr_SVC_auc = OneVsRestClassifier(SVC_auc)

## Make submission

In [ ]:
test_df['clean'] = test_df['comment_text'].apply(simple_clear)
test_df['clean'][:5]

In [ ]:
X_test_sub = word_vectorizer.transform(test_df['clean'])

In [ ]:
log_reg = LogisticRegression(C = 1, penalty='l2', solver = 'liblinear',class_weight = 'balanced', random_state=42)

classifier_ovr_log = OneVsRestClassifier(log_reg)
classifier_ovr_log.fit(X_train_transformed, y_train)

In [ ]:
y_pred = classifier_ovr_log.predict(X_test_sub)
y_pred_df = pd.DataFrame(y_pred,columns=y.columns)
submission_df = pd.concat([test_df.id, y_pred_df], axis=1)
submission_df.to_csv('submission.csv', index = False)